In [ ]:
!pip install gradio ultralytics opencv-python

In [ ]:
!pip install ffmpeg-python

In [ ]:
!apt update && apt install -y ffmpeg

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,651 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,606 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64

In [ ]:
import gradio as gr
import torch
from ultralytics import YOLO
import cv2
import numpy as np
import tempfile
import shutil
import ffmpeg
import os

# Load the trained YOLO segmentation model
MODEL_PATH = "/content/najbolji model.pt"
model = YOLO(MODEL_PATH)

def process_image(image):
    """Perform segmentation on an uploaded image and return the segmented image if precision is above 85%."""
    results = model(image)  # Run YOLO model on the image
    image = np.array(image, dtype=np.uint8)  # Convert to numpy array

    for r in results:
        if r.masks is not None and len(r.masks.data) > 0:
            mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

            for single_mask, score, cls in zip(r.masks.data.cpu().numpy(), r.boxes.conf.cpu().numpy(), r.boxes.cls.cpu().numpy()):
                if score >= 0.85:  # Apply 85% threshold
                    resized_mask = cv2.resize(single_mask, (image.shape[1], image.shape[0]))
                    mask = np.maximum(mask, (resized_mask * 255).astype(np.uint8))

            # Apply colormap to the mask
            mask_colored = cv2.applyColorMap(mask, cv2.COLORMAP_JET)

            # Create an overlay by blending the mask with the original image
            overlay = cv2.addWeighted(image, 0.6, mask_colored, 0.4, 0)

            # Draw bounding boxes & labels with 'Cattle'
            for box, score, cls in zip(r.boxes.xyxy, r.boxes.conf.cpu().numpy(), r.boxes.cls.cpu().numpy()):
                if score >= 0.90:  # Apply threshold
                    x1, y1, x2, y2 = map(int, box)
                    label = "Cattle"
                    cv2.rectangle(overlay, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(overlay, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

            return overlay  # Return the segmented image if confidence >= 85%

    return image  # If no mask detected, return original image

def process_video(video_path):
    """Perform segmentation on an uploaded video and return a compatible video file."""
    cap = cv2.VideoCapture(video_path)
    temp_video_path = tempfile.mktemp(suffix='.avi')  # Save intermediate as AVI
    fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Use XVID for compatibility
    fps = int(cap.get(cv2.CAP_PROP_FPS)) or 30  # Default to 30 FPS if not found
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(temp_video_path, fourcc, fps, (frame_width, frame_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)
        for r in results:
            if r.masks is not None:
                mask = np.zeros((frame.shape[0], frame.shape[1]), dtype=np.uint8)

                for single_mask, score, cls in zip(r.masks.data.cpu().numpy(), r.boxes.conf.cpu().numpy(), r.boxes.cls.cpu().numpy()):
                    if score >= 0.85:
                        resized_mask = cv2.resize(single_mask, (frame.shape[1], frame.shape[0]))
                        mask = np.maximum(mask, (resized_mask * 255).astype(np.uint8))

                mask_colored = cv2.applyColorMap(mask, cv2.COLORMAP_JET)
                overlay = cv2.addWeighted(frame, 0.7, mask_colored, 0.3, 0)

                # Draw bounding boxes & labels with 'Cattle'
                for box, score, cls in zip(r.boxes.xyxy, r.boxes.conf.cpu().numpy(), r.boxes.cls.cpu().numpy()):
                    if score >= 0.90:
                        x1, y1, x2, y2 = map(int, box)
                        label = "Cattle"
                        cv2.rectangle(overlay, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        cv2.putText(overlay, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

                out.write(overlay)
            else:
                out.write(frame)

    cap.release()
    out.release()

    # Convert video to MP4 using FFmpeg
    fixed_video_path = temp_video_path.replace('.avi', '.mp4')
    ffmpeg.input(temp_video_path).output(fixed_video_path, vcodec='libx264', pix_fmt='yuv420p').run(overwrite_output=True)
    os.remove(temp_video_path)  # Remove the intermediate file
    return fixed_video_path

# Gradio UI setup
with gr.Blocks() as iface:
    gr.Markdown("# Cattle Segmentation with YOLO 🐄 🐂 🐏 🐐")
    gr.Markdown("Upload an image or video, and this app will segment and detect cattle.")

    with gr.Tab("Image Processing"):
        img_input = gr.Image(label="Upload Image", type="numpy")
        img_output = gr.Image(label="Segmented Image")
        img_button = gr.Button("Process Image")
        img_button.click(fn=process_image, inputs=img_input, outputs=img_output)

    with gr.Tab("Video Processing"):
        vid_input = gr.File(label="Upload Video")
        vid_output = gr.Video(label="Processed Video")
        vid_button = gr.Button("Process Video")
        vid_button.click(fn=lambda vid: process_video(vid.name) if vid is not None else None, inputs=vid_input, outputs=vid_output)

iface.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://799391615c974ee7dc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



0: 640x384 3 cows, 50.9ms
Speed: 4.4ms preprocess, 50.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 cows, 13.7ms
Speed: 3.4ms preprocess, 13.7ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 cows, 13.8ms
Speed: 3.2ms preprocess, 13.8ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 cows, 13.7ms
Speed: 4.0ms preprocess, 13.7ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 cows, 13.7ms
Speed: 3.2ms preprocess, 13.7ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 cows, 13.8ms
Speed: 3.7ms preprocess, 13.8ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 cows, 14.1ms
Speed: 3.7ms preprocess, 14.1ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 cows, 13.8ms
Speed: 3.6ms preprocess, 13.8ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2088, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1635, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
           ^^^^^^^^^^

WARNING ⚠️ 'source' is missing. Using 'source=/usr/local/lib/python3.11/dist-packages/ultralytics/assets'.

image 1/2 /usr/local/lib/python3.11/dist-packages/ultralytics/assets/bus.jpg: 640x480 1 cow, 1 person, 18.2ms
image 2/2 /usr/local/lib/python3.11/dist-packages/ultralytics/assets/zidane.jpg: 384x640 1 cow, 14.4ms
Speed: 2.1ms preprocess, 16.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2088, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1635, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
           ^^^^^^^^^^

...

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2088, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1635, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
           ^^^^^^^^^^

WARNING ⚠️ 'source' is missing. Using 'source=/usr/local/lib/python3.11/dist-packages/ultralytics/assets'.

image 1/2 /usr/local/lib/python3.11/dist-packages/ultralytics/assets/bus.jpg: 640x480 1 cow, 1 person, 17.2ms
image 2/2 /usr/local/lib/python3.11/dist-packages/ultralytics/assets/zidane.jpg: 384x640 1 cow, 15.5ms
Speed: 6.0ms preprocess, 16.3ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2088, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1635, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
           ^^^^^^^^^^


0: 384x640 2 persons, 14.7ms
Speed: 3.6ms preprocess, 14.7ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 640x384 1 cow, 58.4ms
Speed: 2.8ms preprocess, 58.4ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 cow, 13.7ms
Speed: 2.3ms preprocess, 13.7ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 cow, 14.2ms
Speed: 2.7ms preprocess, 14.2ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 cow, 18.7ms
Speed: 2.6ms preprocess, 18.7ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 cow, 13.7ms
Speed: 2.0ms preprocess, 13.7ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 cow, 13.7ms
Speed: 1.9ms preprocess, 13.7ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 cow, 14.0ms
Speed: 2.5ms preprocess, 14.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.11/dist-packages/starlette/applications.py", line 112, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.11/dist-packages/starlette/middleware/errors.py", line 187, in __call__
    raise exc
  File "/usr/local/lib/python3.11/dist-packages/starlette/middleware/errors.py",